In [1]:
import os
import pandas as pd
import requests
import time
from dotenv import load_dotenv

In [2]:
df = pd.read_csv("ai_news_with_sentiment_scores.csv")

In [3]:
load_dotenv()
SLACK_WEBHOOK_URL = os.getenv("SLACK_WEBHOOK_URL")

#  Function to send Slack alerts
def send_slack_alert(message: str):
    if not SLACK_WEBHOOK_URL:
        print("⚠️ No Slack webhook URL set.")
        return
    try:
        response = requests.post(SLACK_WEBHOOK_URL, json={"text": message}, timeout=10)
        if response.status_code == 200:
            print("✅ Alert sent successfully.")
        else:
            print(f"❌ Failed to send alert: {response.status_code} {response.text}")
    except Exception as e:
        print(f"❌ Error sending alert: {e}")

In [4]:
df.head()

,source,title,description,url,publishedAt,full_text,processed_text,sentiment_score,sentiment
0,"{'id': None, 'name': 'Pypi.org'}",ai-edge-litert-nightly 2.0.4.dev20251013,LiteRT is for mobile and embedded devices.,https://pypi.org/project/ai-edge-litert-nightl...,2025-10-14 11:43:13,ai-edge-litert-nightly 2.0.4.dev20251013 LiteR...,aiedgelitertnightly dev litert mobile embedded...,0.00,Neutral
1,"{'id': None, 'name': 'Ndtvprofit.com'}",HCLTech Q2 Review: AI Led Growth Drives Record...,HCLTech Ltd. delivered a resilient Q2 FY26 per...,https://www.ndtvprofit.com/research-reports/hc...,2025-10-14 11:43:11,HCLTech Q2 Review: AI Led Growth Drives Record...,hcltech q review ai led growth drive record wi...,0.75,Positive
2,"{'id': None, 'name': 'Independent.ie'}",The Big Tech Show: How to use Chat GPT for tot...,What are the best tips for beginners using AI ...,https://www.independent.ie/podcasts/the-big-te...,2025-10-14 11:43:03,The Big Tech Show: How to use Chat GPT for tot...,big tech show use chat gpt total beginner best...,-0.50,Negative
3,"{'id': None, 'name': 'Realstorygroup.com'}",Time to Reconsider Your CMS Platform?,I’m very excited on your behalf about the next...,https://www.realstorygroup.com/Blog/time-recon...,2025-10-14 11:41:11,Time to Reconsider Your CMS Platform? I’m very...,time reconsider cm platform im excited behalf ...,0.60,Positive
4,"{'id': None, 'name': 'Yahoo Entertainment'}",Google says to invest $15 billion in AI data c...,NaN,https://consent.yahoo.com/v2/collectConsent?se...,2025-10-14 11:40:41,Google says to invest $15 billion in AI data c...,google say invest billion ai data centre capac...,0.55,Positive


In [6]:
def main():
    
    
    threshold = 0.7
    
    positive_keywords = ["breakthrough", "innovation", "growth", "partnership", "investment"]
    negative_keywords = ["ban", "crash", "Fraud", "layoffs", "lawsuit", "controversy"]


    df = pd.read_csv("ai_news_with_sentiment_scores.csv", encoding="utf-8")

    # Filter based on sentiment score and keywords
    alerts = df[
        (df["sentiment_score"] > threshold) |
        (df["processed_text"].str.contains('|'.join(positive_keywords + negative_keywords), case=False, na=False))
    ]
    alerts = alerts.reset_index()
    if alerts.empty:
        print("No alerts triggered.")
        return

    # Send Slack alerts
    for i, row in alerts.iterrows():
        title = row.get("title", "Untitled")
        url = row.get("url", "")
        score = row.get("sentiment_score", "N/A")
        sentiment = row.get("sentiment", "Unknown")

        message = (
            f"*News Alert Triggered!*\n"
            f"*Title:* {title}\n"
            f"*Sentiment:* {sentiment} ({score})\n"
            f"*URL:* {url}\n"
        )

        send_slack_alert(message)
        time.sleep(1)

        # Send only first 10 alerts to avoid spamming
        if i >= 10:
            break

if __name__ == "__main__":
    main()


✅ Alert sent successfully.
✅ Alert sent successfully.
✅ Alert sent successfully.
✅ Alert sent successfully.
✅ Alert sent successfully.
✅ Alert sent successfully.
